In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.metrics import roc_curve, auc
import matplotlib
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler

## Popis datasetu 

Využívaná data se týkají přímých marketingových kampaní portugalské banky (Portuguese banking institution). Marketingové kampaně byly realizovány přes telefonní hovory. V několika případech bylo nutné uskutečnit kontakt s klientem vícekrát, pro zjistění, zda došlo k otevření a často bylo třeba provést více než jeden kontakt se stejným klientem, aby bylo možné zjistit, zda byl termínovaný vklad sjednán. 

Zdroj: https://archive.ics.uci.edu/ml/datasets/bank+marketing


In [ ]:
# Načtění csv souboru do pandas DataFrame a výpis prvních 10 řádků

df = pd.read_csv('./data/bank-additional-full.csv', sep=";")

df.head(10)

In [ ]:
# Zobrazení počtu řádků a sloupců
print("Rows count: ", df.shape[0])
print("Columns count: ", df.shape[1])

Datový soubor obsahuje 20 proměnných, 
- 10 numerických 
- 10 kategorických 
a jednu cílovou proměnnou "y", která určuje, zda klient sjednal termínovaný vklad a dosahuje hodnoty "yes" (termínovaný vklad sjednán) a "no" (termínovaný vklad nebyl sjednán).

Soubor obsahuje 41188 pozorování. 

Popis proměnných: 
Numerické proměnné
- 1 - Age: Věk klienta v letech.
- 2 - Balance: Zůstatek na účtu klienta (v eurech).
- 3 - Day: Den v měsíci, kdy byl poslední kontakt s klientem.
- 4 - Duration: Délka posledního telefonického kontaktu (v sekundách).
- 5 - Campaign: Počet kontaktů s klientem během aktuální marketingové kampaně.
- 6 - Pdays: Počet dní od posledního kontaktu (999 znamená, že klient byl nikdy předtím nekontaktován).
- 7 - Previous: Počet kontaktů s klientem před aktuální kampaní.
- 8 - Emp.var.rate: Míra zaměstnanecké variability v posledním čtvrtletí (v procentech).
- 9 - Cons.price.idx: Index cen spotřebitelů (v posledním měsíci).
- 10 - Cons.conf.idx: Index důvěry spotřebitelů (v posledním měsíci).

Kategorické proměnné
- 11 - Job: Typ zaměstnání klienta (např. zaměstnanec, důchodce, podnikatel atd.).
- 12 - Marital: Rodinný stav klienta (např. svobodný, ženatý, rozvedený).
- 13 - Education: Vzdělání klienta (např. základní, střední, vysokoškolské).
- 14 - Default: Zda má klient úvěrové selhání (ano/ne).
- 15 - Housing: Zda má klient hypotéku (ano/ne).
- 16 - Loan: Zda má klient osobní půjčku (ano/ne).
- 17 - Contact: Způsob kontaktu (např. mobilní, pevná linka).
- 18 - Month: Měsíc posledního kontaktu (např. leden, únor atd.).
- 19 - Weekday: Den v týdnu posledního kontaktu (např. pondělí, úterý).
- 20 - Poutcome: Výsledek předchozí marketingové kampaně (např. úspěšná, neúspěšná, žádná).

Cílová proměnná 
Subscription: Binární proměnná indikující, zda klient podepsal termínovaný vklad (ano/ne).
___

## PŘEDZPRACOVÁNÍ DAT

In [ ]:
# Získání souhrnných statistik pro data
df.describe

In [ ]:
df.info()
# Zde vidíme, kolik hodnot a jakého typu proměnná je 

In [ ]:
# Vizualizace cílové proměnné 
plt.figure(figsize=(6, 4))
df['y'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Podepsal klient termínovaný vklad?')
plt.xlabel('y - cílová proměnná  ')
plt.ylabel('Počet klientů')
plt.xticks(rotation=0)
plt.grid(axis='y')

# Zobrazení histogramu
plt.show()

In [ ]:
# Vizualizace vstupujících proměnných 
fig = plt.figure(figsize=(20,20))
cols = list(df.columns)
cols.remove("y")

for i, name in enumerate(cols):
    x = fig.add_subplot(5,4,i+1)
    if (df[name].dtype=="object"):
        x.bar(df[name].sort_values().unique(), df[name].value_counts())
    else:
        x.hist(df[name])
    x.set_title(name)

In [ ]:
# Ošetření chybějících hodnot - je nutné nahradit hodnoty "unknown" hodnotami NA 

df2=df.replace(to_replace="unknown",value=pd.NA)
df2["pdays"]=df2[["pdays"]].replace(to_replace=999,value=np.nan)
df2.head()


In [ ]:
# Nyní můžeme zjistit pro každý sloupec chybějící hodnoty a jejich počet
na=df2.isnull()
na.sum()

In [ ]:
# Korelace 
df_encoded = pd.get_dummies(df, drop_first=True)

# Vytvoření korelační matice
correlation_matrix = df_encoded.corr()

# Vizualizace korelace mezi proměnnými v datasetu pomocí heatmapy
plt.figure(figsize=(30, 30))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Korelační matice')
plt.show()

In [ ]:
# Rozdělení věku do intervalů
age_bins = [17, 29, 39, 49, 64, 98]
age_labels = ['17-29', '30-39', '40-49', '50-64', '65+']
df2['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels)


Na základě informací o datovém souboru a chybějících proměnných jsme se rozhodli z analýzy vyřadit "contact", protože pro výsledek není relevantiní zda byl kontaktován přes pevnou linku nebo mobilní telefon. 
Pro velké množství chybějících pozorování odstraníme také proměnnou "poutcome". 
Pro analýzu je nutné také odstranit proměnnou "nr.employed", protože hodnota je získána až po realizaci telefonního hovoru a ovlivňuje cílovou proměnnou. Pokud dosahuje nr.employed hodnoty 0, pak cílová proměnná = "no". A sloupec age, který je nahrazen novým sloupcem age_bins

In [ ]:
# Odstranění nepotřebných sloupců
df2=df2.drop(["pdays", "duration", "nr.employed","poutcome","contact", "age"],axis=1)
df2.head(10) # Zobrazení dat pro kontrolu

Dataset neobsahuje také úplné informace o čase, pozorování jsou však sesbírána chronologicky. Vytvoříme novou proměnnou "year".  

In [ ]:
# Vytvoření nové proměnné year
df2["year"] = 2008
add_year = 0
months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
actual_month = df2.loc[0, "month"]  

for i in df2.index:
    if (months.index(actual_month) > months.index(df2.loc[i, "month"])):
        add_year += 1  
    df2.loc[i, "year"] += add_year
    actual_month = df2.loc[i, "month"]  

print(df2.head())

In [ ]:
# Vytvoření nových intervalů pro proměnné "previous" a "campaign"

fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

ax1.bar(df["previous"].sort_values().unique(), df["previous"].value_counts())
ax1.set_title('Boxplot of attribute "previous"')
ax2.bar(df["campaign"].sort_values().unique(), df["campaign"].value_counts())
ax2.set_title('Boxplot of attribute "campaign"')

fig.show

In [ ]:
df2["previous_cat"] = pd.cut(df2.previous, bins=[-1, 0, 2, 1000], labels=["0", "1-2", "3+"])
df2["campaign_cat"] = pd.cut(df2.campaign, bins=[-1, 0, 1, 2, 3, 4, 10, 1000], labels=["0", "1", "2", "3", "4", "5-10", "10+"])
#df2.head
# Zobrazení prvních několika řádků s novými kategoriemi
#print(df2[["previous", "previous_cat", "campaign", "campaign_cat"]].head())


In [ ]:
# Standardizace vybraných numerických sloupců
scaler = StandardScaler()

numeric_columns = ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m']

df2[numeric_columns] = scaler.fit_transform(df2[numeric_columns])
 

In [ ]:
 
# Vytvoření kopie datasetu
df_cleaned = df2.copy() 
print(df_cleaned.dtypes)
# Uložení vyčištěného datasetu do CSV souboru
df_cleaned.to_csv("data_cleaned.csv", index=False)

In [ ]:
# Oddělení atributů (features) a cílové proměnné (target)
X = df_cleaned.drop(columns=['y'])  # Všechny sloupce kromě 'y' - vstupní proměnné
y = df_cleaned['y'].values  # Cílová proměnná


In [ ]:
# Vytvoření proměnných feature_cols a odstranění cílové proměnné 'y', aby zůstaly pouze proměnné, které se budou používat jako vstupy do modelu
feature_cols = list(df_cleaned.columns)
feature_cols.remove("y")

In [ ]:
# Rozdělení dat na trénovací a testovací
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 20% testovací, 80% trénovací. Random state znamená, že při každém stuštění bude stejné rozložení dat

# Výpis velikosti trénovací a testovací sady
print("Počet záznamů v trénovací sadě:", X_train.shape[0])
print("Počet záznamů v testovací sadě:", X_test.shape[0])

In [ ]:
# Řešení missing values - nahrazení NaN hodnot nejčastějšími hodnotami (mód)

for col in X_train.columns:
    if X_train[col].dtype == "object" or X_train[col].dtype == "category":
        mode_value = X_train[col].mode()[0]  # Získání módu
        X_train[col] = X_train[col].fillna(mode_value)
 
        

print(X_train)
na3=X_train.isnull()
na3.sum()

In [ ]:
# Řešení missing values - nahrazení NaN hodnot nejčastějšími hodnotami (mód)

for col in X_test.columns:
    if X_test[col].dtype == "object" or X_test[col].dtype == "category":
        mode_value = X_test[col].mode()[0]  # Získání módu
        X_test[col] = X_train[col].fillna(mode_value) 
        

print(X_test)

na4=X_test.isnull()
na4.sum()

In [ ]:
# Převod proměnných typu object nebo category na číselné hodnoty 0 = no, 1 = yes a kategorizace  

# Slovník pro uložení kódovníků
kodovniky = {}

# Funkce pro faktorizaci s ohledem na kódovníky
def factorize_with_dict(df, kodovniky):
    for col in df.columns:
        if df[col].dtype == "object" or df[col].dtype.name == "category":
            if col not in kodovniky:
                # Vytvoření nového kódovníku, pokud neexistuje
                codes, uniques = pd.factorize(df[col])
                kodovniky[col] = dict(enumerate(uniques, 1))  # +1 pro začátek od 1
                df[col] = codes + 1
            else:
                # Použití existujícího kódovníku pro konzistenci
                df[col] = df[col].map({v: k for k, v in kodovniky[col].items()})
        else:
            df[col] = df[col]
    return df

# Faktorizace X_train s uložením kódovníků
X_train = factorize_with_dict(X_train, kodovniky)

# Faktorizace X_test s použitím stejných kódovníků
X_test = factorize_with_dict(X_test, kodovniky)

# Výpis všech kódovníků
for col, kodovnik in kodovniky.items():
    print(f"Kódovník pro sloupec '{col}': {kodovnik}")

In [ ]:
#X_train.head(10)

In [ ]:
#X_test.head(10)

In [ ]:
# Převedení y_train a y_test pro binární hodnoty pomocí np.where
y_train = np.where(y_train == "yes", 1, 0)
y_test = np.where(y_test == "yes", 1, 0)


Vyrovnání tříd (class balancing) na trénovacích datech a srovnání technik: 
- under_sampler - odstraní některé vzorky z většinové třídy, aby se snížil její počet a vyrovnal se s menšinovou třídou
- smote_sampler - vytváří syntetické vzorky pro menšinovou třídu, což zvyšuje její zastoupení
- over_sampler - opakuje vzorky menšinové třídy, čímž zvyšuje jejich počet a vyrovnává třídy

Po srovnání všech výsledků jsme se rozhodly využít over_sampler, který vychází nejlépe.

In [ ]:
 
#under_sampler = RandomUnderSampler(random_state = 42)
 
#X_train2, y_train2 = under_sampler.fit_resample(X_train, y_train)
#print(f"Training target statistics: {Counter(y_train2)}")
#print(f"Testing target statistics: {Counter(y_test)}")
 
#Training target statistics: Counterr({0: 3705, 1: 3705}) 
#Testing target statistics: Counter({0: 7303, 1: 935})


In [ ]:
#SMOTE_sampler = SMOTE(random_state = 42)

#X_train2, y_train2 = SMOTE_sampler.fit_resample(X_train, y_train)
#print(f"Training target statistics: {Counter(y_train2)}")
#print(f"Testing target statistics: {Counter(y_test)}")
 
#Training target statistics: Counter({0: 29245, 1: 29245})
#Testing target statistics: Counter({0: 14623, 1: 1853})

In [ ]:
over_sampler  = RandomOverSampler(random_state=42)
 
X_train2, y_train2 = over_sampler.fit_resample(X_train, y_train)
print(f"Training target statistics: {Counter(y_train2)}")
print(f"Testing target statistics: {Counter(y_test)}")

#Training target statistics: Counter(0): 29245, (1): 29245})
#Testing target statistics: Counter((0): 7303, (1): 935})


___
## MODELOVÁNÍ


MODEL LOGISTICKÉ REGRESE

In [ ]:
# Využíváme data, která jsme získaly z over_sampler
X_train2
y_train2

In [ ]:
# Model logistické regrese s maximálním počtem iterací 10.000
logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train2, y_train2) #Trénování modelu na trénovacích datech
y_pred_logreg = logreg.predict(X_test) # Provedení predikce na základě testovacích dat
y_prob_log = logreg.predict_proba(X_test)[:,1] # Získává pravděpodobnosti pro pozitivní třídu

In [ ]:
# Model logistické regrese s křížovou validací
logreg_cv = LogisticRegressionCV(cv = 5, Cs= 1, scoring = "accuracy", max_iter = 10000)
logreg_cv.fit(X_train2,y_train2) #Trénování modelu na trénovacích datech
y_pred_log_cv = logreg_cv.predict(X_test) # Provedení predikce na základě testovacích dat
y_prob_log_cv = logreg_cv.predict_proba(X_test)[:,1] # Získává pravděpodobnosti pro pozitivní třídu


MODEL ROZHODOVACÍHO STROMU

In [ ]:
# Model rozhodovacího stromu
decision_tree = DecisionTreeClassifier(max_depth = 3)
model_tree = decision_tree.fit(X_train2, y_train2) #Trénování modelu na trénovacích datech
predictions_tree = model_tree.predict(X_test) # Provedení predikce na základě testovacích dat
accuracy_score(y_test, predictions_tree) # Vypočátíní přesnosti modelu

In [ ]:
# Vytvoření slovníku pro uchování feature_name a feature_importance
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(feature_cols, model_tree.feature_importances_):
    feats[feature] = importance #add the name/value pair 

# Měření významnosti jednotlivých proměnných (features) - Gini-importance    
# Vytváří DataFrame pro důležitosti proměnných a vykresluje je jako sloupcový graf
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances.sort_values(by='Gini-importance').plot(kind='bar', rot=45)

In [ ]:
# Model rozhodovacího stromu s křížovou validací 
# Definuje rozsah hyperparametrů pro hledání rozhodovací strom
param_dist = {"max_depth":(3,4,5,6,7,8,9,10),"min_samples_leaf":(1,2,3,4,5),
              "criterion": ["gini", "entropy"]}

# Ladění hyperparametrů pomocí GridSearchCV
model_tree_cv = GridSearchCV(model_tree, param_dist, cv=5)
model_tree_cv.fit(X_test,y_test)

In [ ]:
# Trénování a predikce s nejlepším modelem, který jsme našly na základě kroku výše
model_tree_best = DecisionTreeClassifier( criterion='entropy', max_depth= 4, min_samples_leaf=4) # Nejlepší hyperparametry, které jsme našly pomocí GridSearchCV
model_tree_best.fit(X_train2,y_train2) #Trénování modelu na trénovacích datech
predictions_tree2 = model_tree_best.predict(X_test)
accuracy_score(y_test, predictions_tree2) # Vypočítání přesnosti modelu

In [ ]:
# Manuálně nastavený rozhodovací strom a jeho vizualizace
plt.figure(figsize=(18,18))
plot_tree(model_tree,feature_names = list(X_train2.columns), 
               class_names=["no","yes"],
            filled = True,fontsize=10)  
matplotlib.pyplot.savefig('tree.png')

In [ ]:
# Vizualizace rozhodovacho stromu, který jsme získaly pomocí optimalizace s GridSearchCV
plt.figure(figsize=(18,18))
plot_tree(model_tree_best,feature_names = list(X_train2.columns), 
               class_names=["no","yes"],
            filled = True,fontsize=10)  
matplotlib.pyplot.savefig('tree2.png')

MODEL RANDOM FOREST 

In [ ]:
# Vytvoření modelu náhodného lesa
model_forest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model_forest.fit(X_train2, y_train2) # Trénování modelu na trénovacích datech
predictions_forest = model_forest.predict(X_test) # Provedení predikce na základě testovacích dat
accuracy_score(y_test, predictions_forest) # Vypočítání přesnosti modelu


In [ ]:
# Vytvoření matice záměn pro hodnocení výkonu klasifikačního modelu
confmatrix=confusion_matrix(y_test, predictions_forest)
print(confmatrix)
ConfusionMatrixDisplay.from_estimator(model_forest, X_test, y_test)

Vyhodnocení matice záměn: 
- TN (6.272): Počet případů, kdy model správně předpověděl třídu „ne“ (negativní) a skutečnost byla také „ne“.
- FP (1.031): Počet případů, kdy model nesprávně předpověděl třídu „ano“, ale skutečnost byla „ne“. 
- FN (376): Počet případů, kdy model nesprávně předpověděl třídu „ne“, ale skutečnost byla „ano“. 
- TP (559): Počet případů, kdy model správně předpověděl třídu „ano“ a skutečnost byla také „ano“.

In [ ]:
# Random forest  s křížovou validací
# Ladění nejlepších hyperparametrů pomocí GridSearchCV
parametergrid= {"criterion" : ("gini", "entropy"),"max_depth":(1,2,3,4,5),"min_samples_leaf":(1,2,3,4,5)
}
clf = GridSearchCV(RandomForestClassifier(random_state=42), parametergrid) # Provedení 5ti násobné křížové validace
clf.fit(X_train2, y_train2)
best_model=clf.best_estimator_ # Nejlepší model nalezený metodou GridSearchCV
best_params=clf.best_params_ # Nejlepší parametry tohoto modelu
best_score=clf.best_score_ # Nejlepší dosažené skóre

print("Nejlepší model:", best_model)
print("Nejlepší parametry:", best_params)
print("Nejlepší skóre:", best_score)

In [ ]:
# Trénování a predikce s nejlepším modelem, který jsme našly na základě kroku výše
model_forest_best = RandomForestClassifier( criterion= 'entropy', max_depth= 5, min_samples_leaf=2)
model_forest_best.fit(X_train2,y_train2) # Trénování modelu na trénovacích datech
predictions_forest2 = model_forest.predict(X_test) # Provedení predikce na základě testovacích dat
accuracy_score(y_test, predictions_forest2) # Vypočátíní přesnosti modelu

___
## VYHODNOCENÍ A POROVNÁNÍ MODELŮ

### Porovnání logistické regrese a logistické regrese s křížovou validací

In [ ]:
# Vyhodnocení modelu print("Logistická Regrese:")
print("Přesnost:", accuracy_score(y_test, y_pred_logreg)) # Vypočítá procento správných predikcí z celkového počtu -> přesnost modelu
print(classification_report(y_test, y_pred_logreg)) # Vypočítání metrik - precision, recall, f1-score, support
print("Koeficienty:", logreg.coef_)
print("Intercept:", logreg.intercept_)

# Vyhodnocení modelu print("Logistická RegreseCV:")
print("Přesnost:", accuracy_score(y_test, y_pred_log_cv)) # Vypočítá procento správných predikcí z celkového počtu
print(classification_report(y_test, y_pred_log_cv)) # Vypočítání metrik - precision, recall, f1-score, support
print("Koeficienty:", logreg_cv.coef_) # Výpis koeficientů 
print("Intercept:", logreg_cv.intercept_) # Výpis intercept modelu - hodnota, kde model předpokládá, že výstup bude nula, když jsou všechny vstupní proměnné nulové


### Výsledky logictické regrese
- Přesnost modelu (accuracy): 74 %.
- Precision pro třídu 0 (ne): 95 % (z klientů, které model klasifikoval jako „ne, neuzavřou vklad“, bylo 95 % skutečně správných). Za to přesnost pro třídu 1 byla pouze 26 %. 
- Recall pro třídu 1 (ano): 69 % (model správně rozpoznal 69 % klientů, kteří uzavřeli vklad).
- F1-score: 0.83 pro třídu 0 a 0.37 pro třídu 1, což naznačuje, že model je velmi dobrý v klasifikaci třídy 0, ale slabší v klasifikaci třídy 1.
### Výsledky logictické regrese s křížovou validací
- Přesnost modelu (accuracy): 73 %.
- Precision pro třídu 0 (ne): 95 % (z klientů, které model klasifikoval jako „ne“, bylo 95 % skutečně správných).
- Recall pro třídu 1 (ano): 70 % (model správně rozpoznal 70 % klientů, kteří uzavřeli vklad).
- F1-score: 0.83 pro třídu 0 a 0.37 pro třídu 1, což naznačuje, že model je velmi dobrý v klasifikaci třídy 0, ale slabý v klasifikaci třídy 1.
### Shrnutí
Obě verze modelu vykazují podobné výsledky a dosahují celkové přesnosti 73-74 %. Mají velmi dobrou precision pro třídu 0 - 95 % - dobře předpovádají, že klient neuzavře termínovaný vklad. Recall (citlivost) je u obou modelů 73 - 74 %, což znamená, že modely zachytí skoro tři čtvrtiny všech případů této třídy 0. U třídy 1 jsou modely málo precizní a mají i vysokou míru falešných pozitiv. To znamená obtíže s predikcí klientů, kteří uzavřou vklad a naopak modely lépe klasifikují klienty, kteří vklad neuzavřou. Z hlediska výsledků jsou oba modely skoro totožné, v případě modelu bez křížové validace vychází některé ukazatele lépe jen o 1 %.  

In [ ]:
# Porovnání logistické regrese a logistické regrese s křížovou validací
# Zobrazení intercept a koeficientů
print("Logistická Regrese:")
print("Intercept:", round(logreg.intercept_[0], 3))
for i, name in enumerate(feature_cols):
    print(name,": ",round(logreg.coef_[0,i],3))


print("Logistická RegreseCV:")
print("Intercept:", round(logreg_cv.intercept_[0], 3))
for i, name in enumerate(feature_cols):
    print(name,": ", round(logreg_cv.coef_[0,i],3))

print("\nIndividual fold scores: \n")
for i in range(5):
    print(i+1, ": ", round(logreg_cv.scores_[1][i][0], 5))
    
print ("The average score was: ", round(np.average(logreg_cv.scores_[1]),5))

- Oba modely mají podobné hodnoty interceptu, velice blázko 0. To unamená, že při nulových hodnotách vysvětlujícíh proměnných nemá model vychýlenou pravděpodobnost k třídě 0 nebo 1 ( neuzavření či uzavření vkladu)
- Koeficienty se mezi modely mírně liší. To naznačuje konzistentní vliv jednotlivých proměnných a modely stabilně identifikují podobný vliv pro všechny proměnné, bez ohledu na použití křížové validace
- Výsledky pro jednotlivé foldy a jejich score se pohybují od 0.72294 do 0.72594. To znamená, že model má stabilní výkonnost napříč různými částmi dat.

- **Shrnutí** : Logistická regrese s křížovou validací vykazuje stabilní výkon, ale nepřináší výrazné zlepšení přesnosti oproti běžné logistické regresi. Průměrné skóre v jednotlivých folech ukazuje konzistenci modelu na různých částech dat, což je výhodné pro jeho robustnost. Rozdíly v koeficientech mezi modely jsou minimální, což naznačuje, že vliv jednotlivých proměnných na predikci je obdobný v obou případech.

In [ ]:
# Výpis klíčových metrik a matice záměn pro logistickou regresi
print("Logistic Model - manual")
print("Classification report: \n", classification_report(y_test, y_pred_logreg)) # Výpis klíčových metrik
# Vytvoření matice záměn
print("Confussion matrix") 
mat  =confusion_matrix(y_test, y_pred_logreg)
conf_mat=pd.DataFrame({"True 0":[mat[0][0], mat[1][0]],
                 "True 1":[mat[0][1], mat[1][1]]})
conf_mat.index = ["Predicted 0", "Predicted 1"]
print(conf_mat, "\n")

# Výpis klíčových metrik a matice záměn pro logistickou regresi s křížovou validací
print("Logistic Model - cross validation")
print("Classification report: ")
print(classification_report(y_test, y_pred_log_cv))
print("Confussion matrix")
mat  =confusion_matrix(y_test, y_pred_log_cv)
conf_mat=pd.DataFrame({"True 0":[mat[0][0], mat[1][0]],
                 "True 1":[mat[0][1], mat[1][1]]})
conf_mat.index = ["Predicted 0", "Predicted 1"]
print(conf_mat)

In [ ]:
# ROC křivka
fpr, tpr, thresholds = roc_curve(y_true=y_test, y_score=y_prob_log)
fpr2, tpr2, thresholds2 = roc_curve(y_true=y_test, y_score=y_prob_log_cv)

# Vykreslení ROC křivky
plt.figure(figsize=(10, 6))
plt.plot([0, 1], [0, 1], 'k--', label='Náhodný klasifikátor')
plt.plot(fpr, tpr, label='Manuální hyperparametry')
plt.plot(fpr2, tpr2, label='Křížová validace (best params)')

# Nastavení grafu
plt.xlabel('Falešná pozitivní míra (FPR)')
plt.ylabel('Pravdivá pozitivní míra (TPR)')
plt.title('ROC křivka Random Forest Classifier')
plt.legend()
plt.grid()
plt.show()

Interpretace výsledků ROC křivky pro model logistické regrese

- **Modrá linie** - Představuje model logistické regrese, který byl nastaven s ručně laděnými hyperparametry. Vidíme, že jeho výkon je vyšší než náhodný klasifikátor, protože leží nad černou přerušovanou čarou.
- **Oranžová linie** - Představuje model logistické regrese s hyperparametry optimalizovanými křížovou validací. Oranžová linie je podobná modré, v průběhu místy mírně zaostává oproti modré linii, ale většinou je na podobné úrovni jako modrá linie.
- Oba modely jsou lepší než náhodný klasifikátor (černá přerušovaná čára). Jsou schopny lépe rozlišovat mezi pozitivními a negativními případy než náhodné tipování.
- Původní model logistické regrese si vede o trochu lépe, než laděný model křížovou validací. V tomto případě nevedla křížová validace ke zlepšení výkonu modelu.



___

### Porovnání rozhodovacího stromu a rozhodovacího stromu s křížovou validací

In [ ]:

print("Manuálně nastavený decisiontree \n")
print(classification_report(y_test, predictions_tree))  # Vypočítání metrik - precision, recall, f1-score, support
print(accuracy_score(y_test, predictions_tree), "\n") # Vypočítá procento správných predikcí z celkového počtu
# confusion_matrix - Zobrazuje počet skutečných pozitivních a negativních případů v porovnání s predikcemi.
mat  =confusion_matrix(y_test, predictions_tree)
conf_mat=pd.DataFrame({"True 0":[mat[0][0], mat[1][0]],
                 "True 1":[mat[0][1], mat[1][1]]})
conf_mat.index = ["Predicted 0", "Predicted 1"]
print(conf_mat, "\n")

print("Decision tree pomocí GridSearchCV \n")
print(classification_report(y_test, predictions_tree2))  # Vypočítání metrik - precision, recall, f1-score, support
print(accuracy_score(y_test, predictions_tree2),"\n") # Vypočítá procento správných predikcí z celkového počtu
# confusion_matrix - Zobrazuje počet skutečných pozitivních a negativních případů v porovnání s predikcemi.
mat  =confusion_matrix(y_test, predictions_tree2)
conf_mat=pd.DataFrame({"True 0":[mat[0][0], mat[1][0]],
                 "True 1":[mat[0][1], mat[1][1]]})
conf_mat.index = ["Predicted 0", "Predicted 1"]
print(conf_mat, "\n")

### Výsledky manuálně nastaveného Decision Tree:
- Přesnost (accuracy): 83,10 %.
- Precision pro třídu 0 (ne): 94 % (z klientů, které model klasifikoval jako „ne“, bylo 94 % skutečně správných).
- Recall pro třídu 1 (ano): 59 % (model správně rozpoznal 59 % klientů, kteří uzavřeli vklad s 35% precision).
- F1-score: 0.90 pro třídu 0 a 0.44 pro třídu 1. Model dobře klasifikuje třídu 0, ale má slabší výkon pro třídu 1.

Matice záměn:
- Model správně klasifikoval 6301 případů třídy 0 a 547 případů třídy 1.
- Model chybně klasifikoval 1002 negativních případů jako pozitivní a 388 pozitivních případů jako negativní.

### Výsledky modelu vyladěného pomocí GridSearchCV:
- Přesnost (accuracy): 82,9 % (velmi podobná manuálně nastavenému modelu).
- Precision, recall a F1-score jsou stejné jako u manuálně nastaveného modelu. Liší se pouze recall pro 1. třídu o 1 %.

Matice záměn:
- Model správně klasifikoval 6271 případů třídy 0 a 559 případů třídy 1.
- Chybně klasifikoval 1032 negativních případů jako pozitivní a 376 pozitivních případů jako negativní.
  
### Srovnání výsledků:
- Přesnost (accuracy) obou modelů je téměř totožná: 83,10 % pro manuálně nastavený model a 82,9 % pro GridSearchCV model.
- Precision a recall jsou stejné pro oba modely, což znamená, že ladění hyperparametrů GridSearchCV nepřineslo žádné zlepšení v porovnání s manuálně nastaveným modelem.
- F1-score pro třídu 0 je vysoké (0.90), ale pro třídu 1 je nižší (0.44). To znamená, že modely lépe rozpoznávají třídu 0 (klienty, kteří vklad neuzavřou) než třídu 1.
- GridSearchCV model má mírně lepší schopnost klasifikovat třídu 1 (559 správných predikcí oproti 547 u manuálního modelu), ale zhoršuje se u třídy 0 (6271 správných vs. 6301 u manuálního modelu).
___

### Porovnání manuálně nastaveného Random Forest modelu a modelu, který byl vyladěn pomocí GridSearchCV

In [ ]:
# Porovnání manuálně nastaveného Random Forest modelu a modelu, který byl vyladěn pomocí GridSearchCV
print("Manuálně nastavený Random forest\n")
print(classification_report(y_test, predictions_forest)) # Vypočítání metrik - precision, recall, f1-score, support
print(accuracy_score(y_test, predictions_forest),"\n") # Vypočítá procento správných predikcí z celkového počtu
# confusion_matrix - Zobrazuje počet skutečných pozitivních a negativních případů v porovnání s predikcemi.
mat  =confusion_matrix(y_test, predictions_forest)
conf_mat=pd.DataFrame({"True 0":[mat[0][0], mat[1][0]],
                 "True 1":[mat[0][1], mat[1][1]]})
conf_mat.index = ["Predicted 0", "Predicted 1"]
print(conf_mat, "\n")

print(" Random forest pomocí GridSearchCV \n")
print(classification_report(y_test, predictions_forest2)) # Vypočítání metrik - precision, recall, f1-score, support
print(accuracy_score(y_test, predictions_forest2),"\n") # Vypočítá procento správných predikcí z celkového počtu
# confusion_matrix - Zobrazuje počet skutečných pozitivních a negativních případů v porovnání s predikcemi.
mat  =confusion_matrix(y_test, predictions_forest2)
conf_mat=pd.DataFrame({"True 0":[mat[0][0], mat[1][0]],
                 "True 1":[mat[0][1], mat[1][1]]})
conf_mat.index = ["Predicted 0", "Predicted 1"]
print(conf_mat, "\n")

### Výsledky manuálně nastaveného modelu Random forest :
- Přesnost (accuracy): 82,9 %.
- Precision pro třídu 0 (ne): 94 % (z klientů, které model klasifikoval jako „ne“, bylo 94 % skutečně správných).
- Recall pro třídu 1 (ano): 60 % (model správně rozpoznal 60 % klientů, kteří uzavřeli vklad s 35% precision).
- F1-score: 0.90 pro třídu 0 a 0.44 pro třídu 1 znamená, že model je velmi dobrý v klasifikaci třídy 0, ale slabší v klasifikaci třídy 1.

Matice záměn:
- Model správně předpověděl 6272 negativních případů a 559 pozitivních případů.
- Chybně klasifikoval 1031 negativních případů jako pozitivní a 376 pozitivních případů jako negativní. 

### Výsledky modelu náhodného lesa vyladěného pomocí GridSearchCV:
- Přesnost (accuracy): 82,9 % (stejná jako u manuálního modelu).
- Precision, recall a F1-score jsou také shodné s manuálním modelem.
  
Matice záměn je stejná jako u manuálního modelu a ladění hyperparametrů nemělo žádný dopad na výkonnost modelu.

### Shrnutí:
- Oba modely mají vysokou přesnost při predikci třídy 0 (neuzavření vkladu), ale jsou méně přesné a citlivé při predikci třídy 1 (uzavření vkladu).
- Ladění hyperparametrů pomocí GridSearchCV nemělo v tomto případě žádný vliv na zlepšení výkonu modelu, protože oba modely mají totožné výsledky.

### Závěr:
Všechny modely excelují v klasifikaci třídy 0 (neuzavření vkladu), ale mají problémy s predikcí třídy 1 (uzavření vkladu).
Křížová validace zlepšuje schopnost modelů generalizovat na nových datech, ale v některých případech (např. u náhodného lesa a rozhodovacího stromu) nepřináší žádné zlepšení výkonnosti.
Ladění hyperparametrů pomocí GridSearchCV nemělo v žádném případě zásadní vliv na zlepšení výsledků oproti manuálně nastaveným modelům.
Pro zlepšení výsledků by bylo vhodné přistoupit k pokročilým metodám jako jsou gradient boosting modely nebo modely optimalizovat více na recall pro třídu 1.

